# Reading, Writing and Validating Data in PySpark HW

Welcome to your first coding homework assignment in PySpark! I hope you enjoyed the lecture on Reading, Writing and Validating dataframes. Now it's time to put what you've learned into action! 

I've included several instructions below to help guide you through this homework assignment which I hope will get you feeling even comfortable reading, writing and validating dataframes. If you get stuck at any point, feel free to jump to the next lecture where I will guide you through my solutions to the HW assignment. 

Have fun!

Let's dig right in!


## But first things first.....
We need to always begin every Spark session by creating a Spark instance. Let's go ahead and use the method we learned in the lecture in the cell below. Also see if you can remember how to open the Spark UI (using a link that automatically guides you there). 

In [23]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import *

spark = SparkSession.builder.appName("HW_APP").getOrCreate()

spark

In [24]:
cores = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()
print(cores)

1


## Next let's start by reading a basic csv dataset

Download the pga_tour_historical dataset that is attached to this lecture and save it whatever folder you want, then read it in. 

**Data Source:** https://www.kaggle.com/bradklassen/pga-tour-20102018-data

Rememer to try letting Spark infer the header and infer the Schema types!

In [38]:
path = "/home/iron/Documents/1.Learning/2. PYSPARK_BASICS/Read Write Validate Datasets/datasets/"

pgaTour = spark.read.csv(path+"pgaTourData.csv",inferSchema=True, header=True)

## 1. View first 5 lines of dataframe
First generate a view of the first 5 lines of the dataframe to get an idea of what is inside. We went over two ways of doing this... see if you can remember BOTH ways. 

In [39]:
pgaTour.limit(5).toPandas()

,Player Name,Rounds,Fairway Percentage,Year,Avg Distance,gir,Average Putts,Average Scrambling,Average Score,Points,Wins,Top 10,Average SG Putts,Average SG Total,SG:OTT,SG:APR,SG:ARG,Money
0,Henrik Stenson,60.0,75.19,2018,291.5,73.51,29.93,60.67,69.617,868,NaN,5.0,-0.207,1.153,0.427,0.960,-0.027,"$2,680,487"
1,Ryan Armour,109.0,73.58,2018,283.5,68.22,29.31,60.13,70.758,"1,006",1.0,3.0,-0.058,0.337,-0.012,0.213,0.194,"$2,485,203"
2,Chez Reavie,93.0,72.24,2018,286.5,68.67,29.12,62.27,70.432,"1,020",NaN,3.0,0.192,0.674,0.183,0.437,-0.137,"$2,700,018"
3,Ryan Moore,78.0,71.94,2018,289.2,68.80,29.17,64.16,70.015,795,NaN,5.0,-0.271,0.941,0.406,0.532,0.273,"$1,986,608"
4,Brian Stuard,103.0,71.44,2018,278.9,67.12,29.11,59.23,71.038,421,NaN,3.0,0.164,0.062,-0.227,0.099,0.026,"$1,089,763"


## 2. Print the schema details

Now print the details of the dataframes schema that Spark infered to ensure that it was infered correctly. Sometimes it is not infered correctly, so we need to watch out!

In [40]:
pgaTour.printSchema()

root
 |-- Player Name: string (nullable = true)
 |-- Rounds: double (nullable = true)
 |-- Fairway Percentage: double (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Avg Distance: double (nullable = true)
 |-- gir: double (nullable = true)
 |-- Average Putts: double (nullable = true)
 |-- Average Scrambling: double (nullable = true)
 |-- Average Score: double (nullable = true)
 |-- Points: string (nullable = true)
 |-- Wins: double (nullable = true)
 |-- Top 10: double (nullable = true)
 |-- Average SG Putts: double (nullable = true)
 |-- Average SG Total: double (nullable = true)
 |-- SG:OTT: double (nullable = true)
 |-- SG:APR: double (nullable = true)
 |-- SG:ARG: double (nullable = true)
 |-- Money: string (nullable = true)



## 3. Edit the schema during the read in

We can see from the output above that Spark did not correctly infer that the "value" column was an integer value. Let's try specifying the schema this time to let spark know what the schema should be.

Here is a link to see a list of PySpark data types in case you need it (also attached to the lecture): 
https://spark.apache.org/docs/latest/sql-ref-datatypes.html

In [35]:

from pyspark.sql.types import IntegerType, DataType, DateType, StringType, BooleanType
pgaTour = pgaTour.withColumn("Money", pgaTour.Money.cast(IntegerType()))

pgaTour.printSchema()

root
 |-- Player Name: string (nullable = true)
 |-- Rounds: double (nullable = true)
 |-- Fairway Percentage: double (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Avg Distance: double (nullable = true)
 |-- gir: double (nullable = true)
 |-- Average Putts: double (nullable = true)
 |-- Average Scrambling: double (nullable = true)
 |-- Average Score: double (nullable = true)
 |-- Points: string (nullable = true)
 |-- Wins: double (nullable = true)
 |-- Top 10: double (nullable = true)
 |-- Average SG Putts: double (nullable = true)
 |-- Average SG Total: double (nullable = true)
 |-- SG:OTT: double (nullable = true)
 |-- SG:APR: double (nullable = true)
 |-- SG:ARG: double (nullable = true)
 |-- Money: integer (nullable = true)



## 4. Generate summary statistics for only one variable

See if you can generate summary statistics for only the "Value" column using the .describe function

(count, mean, stddev, min, max) 

In [37]:
pgaTour.describe(["Money"]).show()

+-------+-----+
|summary|Money|
+-------+-----+
|  count|    0|
|   mean| null|
| stddev| null|
|    min| null|
|    max| null|
+-------+-----+



## 5. Generate summary statistics for TWO variables
Now try to generate ONLY the count min and max for BOTH the "Value" and "Season" variable using the select. You can't use the .describe function for this one but see if you can remember which function you CAN use. 

## 6. Write a parquet file

Now try writing a parquet file (not partitioned) from the pga dataset. But first create a new dataframe containing ONLY the the "Season" and "Value" fields (using the "select command you used in the question above) and write a parquet file partitioned by "Season". This is a bit of a challenge aimed at getting you ready for material that will be covered later on in the course. Don't feel bad if you can't figure it out.

*Note that if any of your variable names contain spaces, spark will produce an error message with this call. That is why we are selecting ONLY the "Season" and "Value" fields. Ideally we should renamed those columns but we haven't gotten to that yet in this course but we will soon!*

## 7. Write a partioned parquet file

You will need to use the same limited dataframe that you created in the previous question to accomplish this task as well. 

## 8. Read in a partitioned parquet file

Now try reading in the partitioned parquet file you just created above. 

## 9. Reading in a set of paritioned parquet files

Now try only reading Seasons 2010, 2011 and 2012.

## 10. Create your own dataframe

Try creating your own dataframe below using PySparks *.createDataFrame* function. See if you can make one that contains 4 variables and at least 3 rows. 

Let's see how creative you can get on the content of the dataframe :)

## We're done! Great job!